In [167]:
import pandas as pd
import numpy as np
from urllib import request
from bs4 import BeautifulSoup

import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By

import math
import time

from collections import Counter

import matplotlib.pyplot as plt
import matplotlib
import matplotlib.font_manager as fm
font_loc = 'C:/Windows/Fonts/H2PORM.TTF'
font_name = fm.FontProperties(fname=font_loc).get_name()
matplotlib.rc('font',family=font_name)
plt.rcParams['axes.unicode_minus'] = False

from functools import reduce
import konlpy
from konlpy.tag import Okt

import cx_Oracle
import folium
from folium import plugins
from folium.features import CustomIcon

import requests
import json
from functools import reduce

import webbrowser

import requests, json
from urllib.parse import urlparse

import plotly.graph_objects as go
import plotly.offline as pyo

In [2]:
class CGV:
    
    def __init__(self):
        pass
    
    # CGV 사이트의 현재 상영중인 영화 목록 보여주는 함수
    def show_movie():
        driver = webdriver.Chrome('chromedriver')
        url = 'http://www.cgv.co.kr/movies/?lt=1&ft=1'
        driver.get(url)
        time.sleep(1)  # driver가 url을 받아서 접속하기 전에 html = driver.page_source가 동작하지 않게 하기 위함
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        movie = soup.select('#contents > div.wrap-movie-chart > div.sect-movie-chart')  # 영화 목록을 담고 있는 태그
        m = movie[0].select('.title')  # class가 title인 것만 선택

        movie_list=[]
        for i in range(0,len(m)):
            movie_list.append(m[i].text)

        global df_movie_list_cgv
        df_movie_list_cgv = pd.DataFrame([i for i in movie_list])  # 영화 목록을 데이터 프레임 형태로 저장
        df_movie_list_cgv.columns = ['name']
        
        print(df_movie_list_cgv)
        
        return df_movie_list_cgv
    
    # 사용자가 영화 목록을 보고 영화를 고르면 해당 영화에 대한 url을 제공하는 함수
    def find_url():
        num = int(input('영화의 인덱스를 입력해주세요 : '))

        global name  # global 키워드를 통해 다른 함수에서도 name 변수를 사용할 수 있게 해줌
        name = df_movie_list_cgv.loc[num]['name']

        driver = webdriver.Chrome('chromedriver')
        url = 'http://www.cgv.co.kr/movies/?lt=1&ft=1'
        driver.get(url)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        mv = soup.select('div > a')

        global f_url  # global 키워드를 통해 다른 함수에서도 f_url 변수를 사용할 수 있게 해줌
        f_url = 'http://www.cgv.co.kr' + mv[2+num*2]['href']

        return None
    
    def find_review():
        driver = webdriver.Chrome('chromedriver')
        driver.get(f_url)  # find_url 함수에서 생성한 f_url 변수 사용
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # 영화 댓글 수에 대한 값
        num = soup.select('#menu > div.col-detail > div.sect-grade > div.real-rating > p.desc > span > em')[0]
        n = num.text
        n = n.replace(',','')
        n = int(n)
        n = math.ceil(n / 60)  # 한 페이지에 6개의 댓글이 있고, 한번에 10개의 페이지를 살펴보기 때문에
                               # 60으로 나눈 값을 올림하여서 for문이 도는 횟수를 지정

        comment_list=[]

        # 다음 댓글 페이지로 넘어가는 버튼의 tag 번호 값이 1~10 페이지와 그 이후 페이지가 달라서
        # 둘을 구분하여 만들었다
        # 1~10 페이지까지 댓글 정보
        for i in range(1,11):
            try:
                if (i!=1):
                    btn =  driver.find_element(By.CSS_SELECTOR,f'#paging_point > li:nth-child({i}) > a')
                    btn.click()
                    time.sleep(0.5)  # 버튼이 제대로 눌린 후 다음 명령어가 실행되게 하기 위함
            except:
                break
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            comment = soup.select_one('#movie_point_list_container')
            cm = comment.select('.box-comment')
            for j in range(0,len(cm)):
                comment_list.append(cm[j].text)
        
        chk = 0
        try:
            btn =  driver.find_element(By.CSS_SELECTOR,f'#paging_point > li:nth-child({11}) > button')
            btn.click()
            time.sleep(0.5)
            chk = 1
        except:
            pass

        # 11페이지부터 댓글정보
        if chk == 1:
            for i in range(1,n):
                for j in range(2,11):
                    html = driver.page_source
                    soup = BeautifulSoup(html, 'html.parser')
                    comment = soup.select_one('#movie_point_list_container')
                    cm = comment.select('.box-comment')
                    for k in range(0,len(cm)):
                        comment_list.append(cm[k].text)
                    try:
                        btn =  driver.find_element(By.CSS_SELECTOR,f'#paging_point > li:nth-child({j+2}) > a')
                        # 1~10 페이지와 달리 이전 페이지로 넘어가는 버튼 2개가 추가되어서
                        # j가 아닌 j+2를 사용해준다
                        btn.click()
                        time.sleep(0.5)
                    except:
                        break
                try:        
                    # 다음 10개의 페이지로 넘어가는 버튼
                    btn =  driver.find_element(By.CSS_SELECTOR,f'#paging_point > li:nth-child({13}) > button')
                    btn.click()
                    time.sleep(0.5)
                except:
                    break

        global df_dict_cgv
        df_dict_cgv = dict()        
        
        # 댓글이 있다면 dict에 저장하고 없다면 '댓글이 없습니다' 출력
        if len(comment_list) > 0:
            df = pd.DataFrame([i for i in comment_list])
            df.columns = ['comment']
            df_dict_cgv[name]=df
            print(df_dict_cgv[name])
            return df_dict_cgv[name]
        else:
            print('댓글이 없습니다')
            return None
        
    # cgv 영화 댓글에 대한 감정 그래프를 그려주는 함수
    def emotion():
        nlp=Okt()
        result = []
        
        def review_noun(a): # 명사만 추출하는 함수
            result = []
            for i in a:
                temp = []
                for j in i:
                    try:
                        for k in nlp.nouns(j):
                            if len(k)<=1:
                                continue
                            temp.append(k)
                    except:
                        pass
                result.append(temp)
            return result

        a = df_dict_cgv[name].iloc[:,0].apply(lambda x:x.split()) # 먼저 split으로 띄어쓰기로 분리
        result = review_noun(a)
        
        sad = 0
        emp = 0
        con = 0
        rel = 0
        fun = 0
        
        # 감동, 공감, 몰입감, 스트레스해소, 유쾌함 5가지 키워드에 대한 단어 리스트를 만들고
        # 댓글과 단어 리스트를 비교하여 교집합이 하나라도 있다면 해당 키워드에 대한 count를 +1 해주었다
        for i in range(0,len(result)):
            if len(sadList&set(result[i])) >= 1:
                sad += 1
            if len(empList&set(result[i])) >= 1:
                emp += 1
            if len(conList&set(result[i])) >= 1:
                con += 1
            if len(relList&set(result[i])) >= 1:
                rel += 1
            if len(funList&set(result[i])) >= 1:
                fun += 1
                
        c_list = []
        c_list.append(sad)
        c_list.append(emp)
        c_list.append(con)
        c_list.append(rel)
        c_list.append(fun)

        c_list = [*c_list, c_list[0]]

        categories =  ['감동','공감','몰입감','스트레스해소','유쾌함']
        categories = [*categories, categories[0]]
        
        # label이 뜨는 위치를 지정
        label_loc = [np.pi/2-2*np.pi/5,np.pi/2,np.pi/2+2*np.pi/5,np.pi/2+4*np.pi/5,np.pi/2+6*np.pi/5,np.pi/2-2*np.pi/5]

        plt.figure(figsize=(8,8))
        ax = plt.subplot(polar=True)
        plt.xticks(label_loc, labels=categories, fontsize=13)
        ax.plot(label_loc, c_list, label='감정 그래프', color='violet')
        ax.fill(label_loc, c_list, color='violet', alpha=0.3)
        ax.legend()
        plt.show()

In [173]:
class MEGA:
    
    def __init__(self):
        pass
    
    # Megabox의 현재 상영중인 영화 목록을 보여주는 함수
    def show_movie():
        driver = webdriver.Chrome('chromedriver')
        # 다른 사이트들과 다르게 메인 화면에서 영화 목록을 호출 했을 때 빈 리스트가 호출되어서
        # 메가박스 상영시간표 url에서 영화 목록을 가져왔다
        url = 'https://www.megabox.co.kr/booking/timetable'
        driver.get(url)
        time.sleep(1)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        movie = soup.select('#mCSB_2_container > ul')
        m = movie[0].select('button.btn')

        movie_list=[]
        for i in range(0,33):
            movie_list.append(m[i].text)

        global df_movie_list_megabox
        df_movie_list_megabox = pd.DataFrame([i for i in movie_list])
        df_movie_list_megabox.columns = ['name']
        
        print(df_movie_list_megabox)

        return df_movie_list_megabox

    # 사용자가 영화 목록을 보고 영화 인덱스를 선택했을 때 해당 영화에 대한 url을 찾아주는 함수
    def find_url():
        num = int(input('영화의 인덱스를 입력해주세요 : '))

        global name
        name = df_movie_list_megabox.loc[num]['name']

        driver = webdriver.Chrome('chromedriver')
        url = 'https://www.megabox.co.kr/booking/timetable'
        driver.get(url)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        mv = soup.select('#mCSB_2_container > ul')

        global f_url
        f_url = 'https://www.megabox.co.kr/movie-detail/comment?rpstMovieNo=' + mv[0].select('button.btn')[num]['data-movie-no']

        return None

    # 메가박스 영화 댓글을 찾아주는 함수
    def find_review(): # name: 영화 이름으로 데이터프레임 dict 안에 넣음 / 메가박스용
        driver = webdriver.Chrome('chromedriver')
        driver.get(f_url)
        soup = BeautifulSoup(driver.page_source,'html.parser')
        result = []
        result_charm = []
        result2 = []
        result2_charm = []
        for i in range(10): # 10페이지 까지
            try:
                if (i!=0):
                    btn = driver.find_element(By.CSS_SELECTOR,f'#contentData > div > div.movie-idv-story > nav > a:nth-child({i+1})')# 버튼 생성
                    btn.click()  # 클릭
                    time.sleep(0.5)
            except:
                break
            soup = BeautifulSoup(driver.page_source,'html.parser')
            page = soup.select_one('#contentData > div > div.movie-idv-story > ul')
            result.append(page.findAll('div',{'class':'story-txt'}))
            result_charm.append(page.findAll('div',{'class':'story-recommend'}))
        
        try:
            btn = driver.find_element(By.CSS_SELECTOR,'#contentData > div > div.movie-idv-story > nav > a.control.next') 
            btn.click()
            time.sleep(0.5)
        except:
            pass
        
        for i in range(11,math.ceil(int(driver.find_element(By.CSS_SELECTOR,'#cnt1').text.replace(',',''))/10)+1): 
            # 총 개수 / 10 한 몫만큼의 페이지 존재, 검색 지금은 2~82페이지
            try:
                if (i%10==0 and i>0):
                    btn = driver.find_element(By.CSS_SELECTOR,f'#contentData > div > div.movie-idv-story > nav > a:nth-child({12})')# 버튼 생성
                    btn.click()  # 클릭
                    time.sleep(0.5)
                    
                    soup = BeautifulSoup(driver.page_source,'html.parser')
                    page = soup.select_one('#contentData > div > div.movie-idv-story > ul')
                    result2.append(page.findAll('div',{'class':'story-txt'}))
                    result2_charm.append(page.findAll('div',{'class':'story-recommend'}))

                    btn = driver.find_element(By.CSS_SELECTOR,'#contentData > div > div.movie-idv-story > nav > a.control.next') # 10도달 시 다음
                    btn.click()
                    time.sleep(0.5)
                elif (i%10!=1):
                    btn = driver.find_element(By.CSS_SELECTOR,f'#contentData > div > div.movie-idv-story > nav > a:nth-child({i%10+2})')# 버튼 생성
                    btn.click()  # 클릭
                    time.sleep(0.5)

                    soup = BeautifulSoup(driver.page_source,'html.parser')
                    page = soup.select_one('#contentData > div > div.movie-idv-story > ul')
                    result2.append(page.findAll('div',{'class':'story-txt'}))
                    result2_charm.append(page.findAll('div',{'class':'story-recommend'}))
                elif (i%10==1):
                    soup = BeautifulSoup(driver.page_source,'html.parser')
                    page = soup.select_one('#contentData > div > div.movie-idv-story > ul')
                    result2.append(page.findAll('div',{'class':'story-txt'}))
                    result2_charm.append(page.findAll('div',{'class':'story-recommend'}))
            except:
                break

        result3 = result+result2 # 두 결과 합치기(댓글)
        result_charm3 = result_charm + result2_charm # 두 결과 합치기(매력)
    
        charm3= []
        for i in result_charm3: # 매력 포인트 댓글 별로 나누기(두 개 이상 입력한 경우가 있어 적용)
            for j in i:
                if len(j)>2:
                    temp = []
                    for k in j.findAll('em'):
                        temp.append(k.text)
                    charm3.append(temp)
                else:
                    charm3.append(j.text)

        from functools import reduce
        result4 = reduce(lambda x,y:x+y,result3) # 댓글 내 리스트 전부 합치기

        df = pd.DataFrame(data=(result4,charm3)).T # 데이터 프레임 생성
        df.columns = ['comment', 'charm'] # 댓글과 매력 포인트

        global df_dict_mega
        df_dict_mega = dict()

        df_dict_mega[name]=df
        
        if len(df_dict_mega[name]) == 0:
            print('댓글이 없습니다')
        else:
            print(df_dict_mega[name])
            
    # 메가박스 사이트에는 각 댓글마다 댓글 작성자가 해당 영화에 대한 매력 포인트를 선택한 것이 존재한다
    # 매력 포인트들을 count 하여 해당 영화에 대한 매력 포인트를 그려주는 함수
    # 감정 포인트 그래프와 다른 방식을 사용하였다 (plotly.graph_objects, plotly.offline 사용)
    def charm():
        charm = []
        for i in df_dict_mega[name].iloc[:,1].tolist():
            if type(i)==list:
                for j in i:
                    charm.append(j.strip())
            else:
                charm.append(i.strip())
        charm_count = Counter(charm)
        
        c_list = []
        c_list.append(charm_count['배우'])
        c_list.append(charm_count['연출'])
        c_list.append(charm_count['스토리'])
        c_list.append(charm_count['영상미'])
        c_list.append(charm_count['OST'])
        
        c_list = [*c_list, c_list[0]]
        
        categories =  ['배우','연출','스토리','영상미','OST']
        categories = [*categories, categories[0]]

        fig = go.Figure(
        data =[
            go.Scatterpolar(r=c_list, theta=categories, fill ='toself', name='매력그래프')
        ],
        layout=go.Layout(
        title=go.layout.Title(text='매력그래프'),
        polar={'radialaxis': {'visible': True}},
        showlegend=True))
        pyo.plot(fig)
        
    # 메가박스 영화 댓글에 대한 감정 포인트 그래프 그려주는 함수
    def emotion():
        nlp=Okt()
        result = []
        
        def review_noun(a): # 명사만 추출하는 함수
            result = []
            for i in a:
                temp = []
                for j in i:
                    try:
                        for k in nlp.nouns(j):
                            if len(k)<=1:
                                continue
                            temp.append(k)
                    except:
                        pass
                result.append(temp)
            return result
        
        cml = []
        for i in range(0,len(df_dict_mega[name])):
            cml.append(df_dict_mega[name].iloc[:,0][i].text)
            
        cmdf = pd.DataFrame(data=cml)
        
        a = cmdf.iloc[:,0].apply(lambda x:x.split()) # 먼저 split으로 띄어쓰기로 분리
        result = review_noun(a)
        
        sad = 0
        emp = 0
        con = 0
        rel = 0
        fun = 0
        
        # CGV와 동일
        for i in range(0,len(result)):
            if len(sadList&set(result[i])) >= 1:
                sad += 1
            if len(empList&set(result[i])) >= 1:
                emp += 1
            if len(conList&set(result[i])) >= 1:
                con += 1
            if len(relList&set(result[i])) >= 1:
                rel += 1
            if len(funList&set(result[i])) >= 1:
                fun += 1
                
        c_list = []
        c_list.append(sad)
        c_list.append(emp)
        c_list.append(con)
        c_list.append(rel)
        c_list.append(fun)

        c_list = [*c_list, c_list[0]]

        categories =  ['감동','공감','몰입감','스트레스해소','유쾌함']
        categories = [*categories, categories[0]]

        label_loc = [np.pi/2-2*np.pi/5,np.pi/2,np.pi/2+2*np.pi/5,np.pi/2+4*np.pi/5,np.pi/2+6*np.pi/5,np.pi/2-2*np.pi/5]

        plt.figure(figsize=(8,8))
        ax = plt.subplot(polar=True)
        plt.xticks(label_loc, labels=categories, fontsize=13)
        ax.plot(label_loc, c_list, label='감정 그래프', color='violet')
        ax.fill(label_loc, c_list, color='violet', alpha=0.3)
        ax.legend()
        plt.show()

In [174]:
class LOTTE:
    
    def __init__(self):
        pass
    
    # 롯데 시네마의 현재 상영중인 영화의 목록을 보여주는 함수
    def show_movie():
        driver = webdriver.Chrome('chromedriver')
        url = 'https://www.lottecinema.co.kr/NLCHS/Movie/List?flag=1'
        driver.get(url)
        time.sleep(1)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        movie = soup.select('#contents > div')
        m = movie[0].select('strong.tit_info')

        movie_list=[]
        for i in range(0,len(m)):
            movie_list.append(m[i].text)

        global df_movie_list_lotte
        df_movie_list_lotte = pd.DataFrame([i for i in movie_list])
        df_movie_list_lotte.columns = ['name']
        
        print(df_movie_list_lotte)

        return df_movie_list_lotte
    
    # 사용자가 선택한 영화에 대한 url을 호출하는 함수
    def find_url():
        num = int(input('영화의 인덱스를 입력해주세요 : '))

        global name
        name = df_movie_list_lotte.loc[num]['name']

        driver = webdriver.Chrome('chromedriver')
        url = 'https://www.lottecinema.co.kr/NLCHS/Movie/List?flag=1'
        driver.get(url)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        movie = soup.select('#contents > div')
        global f_url
        f_url = movie[0].select('a.btn_col3')[num*2+1]['href']

        return None
    
    # 롯데 시네마의 영화 댓글을 호출하는 함수
    def find_review():
        driver = webdriver.Chrome('chromedriver')
        driver.get(f_url)
        time.sleep(0.5)
        try:
            btn = driver.find_element(By.CSS_SELECTOR,f'#contents > ul > li:nth-child(2) > button')
            btn.click()
        except:
            pass
            
        while(True):
            try:
                btn =  driver.find_element(By.CSS_SELECTOR,'#btn_review_more')
                btn.click()
                time.sleep(0.5)
            except:
                break
        html =driver.page_source
        soup = BeautifulSoup(driver.page_source,'html.parser')

        comment = soup.select('#review_con_list2')
        i = 0
        comment_list = []
        star_list =[]
        while(True):
            try:
                comment_list.append(comment[0].select('.review_info')[i].text)
                star_list.append(comment[0].select('span.gradenum')[i].text)
                i += 1
            except:
                break
        df = pd.DataFrame(columns =['comment','star']) # 댓글과 별점
        df['comment'] = [i for i in comment_list]
        df['star'] = [i for i in star_list]
        global df_dict_lotte
        df_dict_lotte = dict()
        df_dict_lotte[name]=df
        
        if len(df_dict_lotte[name]) == 0:
            print('댓글이 없습니다')
        else:
            print(df_dict_lotte[name])
        
        return df_dict_lotte[name]
    
    # 롯데 시네마의 영화 댓글에 대한 감정 포인트 그래프를 그려주는 함수
    def emotion():
        nlp=Okt()
        result = []
        
        def review_noun(a): # 명사만 추출하는 함수
            result = []
            for i in a:
                temp = []
                for j in i:
                    try:
                        for k in nlp.nouns(j):
                            if len(k)<=1:
                                continue
                            temp.append(k)
                    except:
                        pass
                result.append(temp)
            return result

        a = df_dict_lotte[name].iloc[:,0].apply(lambda x:x.split()) # 먼저 split으로 띄어쓰기로 분리
        result = review_noun(a)
        
        sad = 0
        emp = 0
        con = 0
        rel = 0
        fun = 0
        
        # CGV와 동일
        for i in range(0,len(result)):
            if len(sadList&set(result[i])) >= 1:
                sad += 1
            if len(empList&set(result[i])) >= 1:
                emp += 1
            if len(conList&set(result[i])) >= 1:
                con += 1
            if len(relList&set(result[i])) >= 1:
                rel += 1
            if len(funList&set(result[i])) >= 1:
                fun += 1
                
        c_list = []
        c_list.append(sad)
        c_list.append(emp)
        c_list.append(con)
        c_list.append(rel)
        c_list.append(fun)

        c_list = [*c_list, c_list[0]]

        categories =  ['감동','공감','몰입감','스트레스해소','유쾌함']
        categories = [*categories, categories[0]]

        label_loc = [np.pi/2-2*np.pi/5,np.pi/2,np.pi/2+2*np.pi/5,np.pi/2+4*np.pi/5,np.pi/2+6*np.pi/5,np.pi/2-2*np.pi/5]

        plt.figure(figsize=(8,8))
        ax = plt.subplot(polar=True)
        plt.xticks(label_loc, labels=categories, fontsize=13)
        ax.plot(label_loc, c_list, label='감정 그래프', color='violet')
        ax.fill(label_loc, c_list, color='violet', alpha=0.3)
        ax.legend()
        plt.show()

In [175]:
class DAUM:
    
    def __init__(self):
        pass
    
    # 다음 영화 사이트에 있는 현재 상영중 영화 목록을 호출하는 함수
    def show_movie():
        driver = webdriver.Chrome('chromedriver')
        url = 'https://movie.daum.net/ranking/reservation'
        driver.get(url)
        time.sleep(1)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        movie = soup.select('#mainContent > div > div.box_ranking')
        m = movie[0].select('a.link_txt')

        movie_list=[]
        for i in range(0,len(m)):
            movie_list.append(m[i].text)

        global df_movie_list_daum
        df_movie_list_daum = pd.DataFrame([i for i in movie_list])
        df_movie_list_daum.columns = ['name']
        
        print(df_movie_list_daum)
        
        return df_movie_list_daum
    
    # 사용자가 선택한 영화에 대한 다음 영화 url을 호출하는 함수
    def find_url():
        num = int(input('영화의 인덱스를 입력해주세요 : '))

        global name
        name = df_movie_list_daum.loc[num]['name']

        driver = webdriver.Chrome('chromedriver')
        url = 'https://movie.daum.net/ranking/reservation'
        driver.get(url)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        mv = soup.select('div > a')

        global f_url
        f_url = 'https://movie.daum.net' + mv[7+num]['href']
        f_url = f_url.replace('main','grade')

        return None
    
    # 해당 영화에 대해 다음 영화 댓글을 호출하는 함수
    def find_review():
        driver = webdriver.Chrome('chromedriver')
        driver.get(f_url)
        time.sleep(0.5)
        while(True):
            try:
                btn =  driver.find_element(By.CSS_SELECTOR,'#alex-area > div > div > div > div.cmt_box > div.alex_more > button')
                btn.click()
                time.sleep(0.5)
            except:
                break
        html = driver.page_source
        soup = BeautifulSoup(driver.page_source,'html.parser')

        comment = soup.select('#alex-area > div > div > div > div.cmt_box > ul.list_comment')

        i = 0
        comment_list = []
        star_list=[]
        while(True):
            try:
                comment_list.append(comment[0].select('p.desc_txt')[i].text)   # p class 라서 p. class 그리고 desc_txt 한칸 띄고 더
                                                                               # 있는 경우 앞에 desc_txt 까지만 쓰기
                star_list.append(int(comment[0].select('.ratings')[i].text))
                i += 1
            except:
                break

        df = pd.DataFrame(columns=['comment','score']) # 댓글과 점수
        df['comment'] = [i for i in comment_list]
        df['score'] = [i for i in star_list]
        global df_dict_daum
        df_dict_daum = dict()
        df_dict_daum[name]=df
        
        if len(df_dict_daum[name]) == 0:
            print('댓글이 없습니다')
        else:
            print(df_dict_daum[name])
        
        return df_dict_daum
    
    # 다음 영화 댓글에 대한 감정 포인트 그래프를 그려주는 함수
    def emotion():
        nlp=Okt()
        result = []
        
        def review_noun(a): # 명사만 추출하는 함수
            result = []
            for i in a:
                temp = []
                for j in i:
                    try:
                        for k in nlp.nouns(j):
                            if len(k)<=1:
                                continue
                            temp.append(k)
                    except:
                        pass
                result.append(temp)
            return result

        a = df_dict_daum[name].iloc[:,0].apply(lambda x:x.split()) # 먼저 split으로 띄어쓰기로 분리
        result = review_noun(a)
        
        sad = 0
        emp = 0
        con = 0
        rel = 0
        fun = 0
        
        # cgv와 동일
        for i in range(0,len(result)):
            if len(sadList&set(result[i])) >= 1:
                sad += 1
            if len(empList&set(result[i])) >= 1:
                emp += 1
            if len(conList&set(result[i])) >= 1:
                con += 1
            if len(relList&set(result[i])) >= 1:
                rel += 1
            if len(funList&set(result[i])) >= 1:
                fun += 1
                
        c_list = []
        c_list.append(sad)
        c_list.append(emp)
        c_list.append(con)
        c_list.append(rel)
        c_list.append(fun)

        c_list = [*c_list, c_list[0]]

        categories =  ['감동','공감','몰입감','스트레스해소','유쾌함']
        categories = [*categories, categories[0]]

        label_loc = [np.pi/2-2*np.pi/5,np.pi/2,np.pi/2+2*np.pi/5,np.pi/2+4*np.pi/5,np.pi/2+6*np.pi/5,np.pi/2-2*np.pi/5]

        plt.figure(figsize=(8,8))
        ax = plt.subplot(polar=True)
        plt.xticks(label_loc, labels=categories, fontsize=13)
        ax.plot(label_loc, c_list, label='감정 그래프', color='violet')
        ax.fill(label_loc, c_list, color='violet', alpha=0.3)
        ax.legend()
        plt.show()

In [176]:
# Movie 클래스에 CGV, MEGA, LOTTE, DAUM 클래스 상속
# CGV, MEGA, LOTTE, DAUM 클래스의 함수 이름은 되도록이면 동일하게
# CGV, MEGA, LOTTE, DAUM 클래스의 각 기능은 서로 다를 수 있다
class Movie(CGV,MEGA,LOTTE,DAUM):
    
    # 메뉴 화면을 따로 변수로 설정하여 영화관 브랜드 별로 관리
    # 한 영화관 브랜드에서 새로 기능을 추가하더라도 해당 영화관 브랜드의 메뉴 변수만 바꿔주면 된다
    
    # 전체 메뉴
    global mn
    mn = '''-----------------------------------------------------
                    1. CGV 정보                      
                  2. MEGABOX 정보   
                3. LOTTE CINEMA 정보
                 4. DAUM 영화 정보
                      5. 종료                      
-----------------------------------------------------'''
    
    # CGV 메뉴
    global m_CGV
    m_CGV = '''-----------------------------------------------------
                        CGV
                        
          1. 현재 상영중인 영화 목록 보기               
                   2. 지도 보기 
                    3. 나가기                       
-----------------------------------------------------'''
    
    # 메가박스 메뉴
    global m_Mega
    m_Mega = '''-----------------------------------------------------
                      MEGABOX
          
          1. 현재 상영중인 영화 목록 보기               
                   2. 지도 보기 
                    3. 나가기                       
-----------------------------------------------------'''
    
    # 롯데 시네마 메뉴
    global m_Lotte
    m_Lotte = '''-----------------------------------------------------
                   LOTTE CINEMA
          
          1. 현재 상영중인 영화 목록 보기               
                   2. 지도 보기 
                    3. 나가기                       
-----------------------------------------------------'''
    
    # 다음 영화 메뉴
    global m_Daum
    m_Daum = '''-----------------------------------------------------
                     DAUM 영화
          
          1. 현재 상영중인 영화 목록 보기               
                    2. 나가기                        
-----------------------------------------------------'''
    
    # 세부 메뉴1 (메가박스만 해당)
    global mini_m1
    mini_m1 = '''-----------------------------------------------------
       1. 영화에 대한 매력포인트 그래프 보기
       2. 영화에 대한 감정포인트 그래프 보기
                    3. 나가기
-----------------------------------------------------'''
    
    # 세부 메뉴2 (메가박스를 제외한 다른 영화관 브랜드에 해당)
    global mini_m2
    mini_m2 = '''-----------------------------------------------------
       1. 영화에 대한 감정포인트 그래프 보기
                    2. 나가기
-----------------------------------------------------'''
    
    # CGV, 메가박스, 롯데시네마 세 곳에서 공통되고 인기 많은 영화 5개 
    # (공조2, 인생은 아름다워, 짱구, 정직한 후보2, 아바타) 의 댓글을 명사 단위로 분리한 후
    # 그 명사들을 count하여 개수가 많은 100개 명사를 추출하였다
    # 각 영화마다 추출한 명사에 대한 리스트를 만들고 그 리스트들에 대해서 서로 비교하여 공통된 명사만 따로 추출하였다
    # 그 명사들에 대해 감정에 따라서 분류하였다
    global sadList
    sadList = set(['가슴', '가족', '감동', '감성', '감정', '기분', '눈물', '마음', '부모님', '사랑', '슬픔', '우정', '펑펑', '휴지', '힐링'])
    global empList
    empList = set(['가족', '감성', '공감', '교훈', '생각', '시절', '여운', '옛날', '웃기', '웃음', '유머', '이야기', '장면', '전개', '진실','첫사랑', '청춘', '추억', '친구', '현실'])
    global conList
    conList = set(['개연', '노래', '몰입', '배우','스릴', '스케일', '스토리', '액션', '연기', '연기력', '연출', '음악', '이야기', '전개', '추리', '케미'])
    global relList
    relList = set(['꿀잼', '대박', '스릴', '액션', '웃기', '웃음', '유머', '재미', '재미나', '재밋', '존잼'])
    global funList
    funList = set(['개그', '꿀잼', '대박', '오락', '웃기', '웃음', '유머', '재미', '재미나', '재밋', '존잼', '코미디', '코믹', '풍자'])
    
    def __init__(self):
        pass
    
    # map m에 origin~destination까지의 경로 표시
    def navigate(origin_lat,origin_lng,destination_lat,destination_lng,m):
        REST_API_KEY='18737d5fc137442681e6094df137a2eb'
        h = { "Authorization": f"KakaoAK {REST_API_KEY}",'Content-Type': 'application/json' }
        url = f"https://apis-navi.kakaomobility.com/v1/directions?origin={origin_lng},{origin_lat}&destination={destination_lng},{destination_lat}&waypoints=&priority=RECOMMEND&car_fuel=GASOLINE&car_hipass=false&alternatives=false&road_details=false"
        a = requests.get(url,headers=h).text
        b=json.loads(a)
        c=b['routes'][0]['sections'][0]['roads']
        d = reduce(lambda x,y:x+y,[i['vertexes'] for i in c])
        result = [d[4*i:4*i+2] for i in range(len(d)//4)]+[[d[-2],d[-1]]]
        result = [i[::-1] for i in result]

        folium.PolyLine(locations=result,tooltip='거리: '+str(round(b['routes'][0]['summary']['distance']/1000,1))
                    +'km 소요시간: '+str(b['routes'][0]['summary']['duration']//60)+'분').add_to(m)
    
    # 사용자가 직접 위도, 경도를 입력하지 않고 주소만 입력하여
    # 카카오 api를 통해 위도, 경도 값을 추출
    def curr_loc(loc):
        try:
            REST_API_KEY = '18737d5fc137442681e6094df137a2eb'
            url = "https://dapi.kakao.com/v2/local/search/address.json"
            h = { "Authorization": f"KakaoAK {REST_API_KEY}",'Content-Type': 'application/json' }
            param = {"query":loc} 
            re = requests.get(url,headers = h, params = param).json()
            
            # 사용자가 올바르지 않은 주소를 입력했을 때 확인하기 위한 변수
            global check
            check = 0
            
            #위도 경도
            global num_lat
            num_lat = re['documents'][0]['y']
            global num_lng
            num_lng = re['documents'][0]['x']
            
            return re['documents'][0]['y'],re['documents'][0]['x'] ,re['documents'][0]['road_address']['building_name']
        except Exception as e:
            check = 1 # 올바르지 않은 주소 입력으로 인해 에러가 발생했을 때 check 값을 1로 바꿔준다
            return np.NaN,np.NaN,np.NaN
    
    # 실제 사용자가 실행하는 함수
    # 사용자가 입력하는 값에 따라 CGV, MEGA, LOTTE, DAUM 클래스에서 필요한 함수를 호출하여 기능 수행
    def menu(self):
        con = cx_Oracle.connect('SCOTT','TIGER',dsn='127.0.0.1:1521/xe',encoding='UTF-8')
        cursor =  con.cursor()  # 쿼리를 실행하는 편집창
        while(True):
            print(mn)
            num = int(input('번호를 입력하세요 : '))
        
            if num == 1:
                while(True):
                    print(m_CGV)
                    num_CGV = int(input('번호를 입력하세요 : '))
                    if num_CGV == 1:
                        CGV.show_movie()                                  # 1. 우선, 영화 목록을 보여주고
                        yn = input('영화 댓글 정보를 확인하시겠습니까? (y/n) : ')
                        if yn == 'y':
                            CGV.find_url()                                # 2. 영화를 선택하여 url을 찾아준다
                            CGV.find_review()                             # 3. 찾은 url을 통해 댓글을 찾아서 dict에 저장
                            while(True):
                                print(mini_m2)
                                choice_CGV = int(input('번호를 입력하세요 : '))
                                if choice_CGV == 1:
                                    CGV.emotion()                         # 4. 필요하다면 감정 포인트 그래프를 그려준다
                                elif choice_CGV == 2:
                                    break
                                    
                    elif num_CGV == 2:
                        while(True):
                            loc = input('현재 위치를 입력하세요 : ')
                            Movie.curr_loc(loc)
                            if check == 1: # check 값이 1이라면 while문을 탈출하지 않고 다시 위치를 입력받는다
                                print('위치 정보가 올바르지 않습니다')
                            elif check == 0: # check 값이 0이라면 while문을 탈출하여 아래 명령문 실행
                                break
                        
                        place = input('방문할 CGV 지점 이름을 입력하세요 : ')
                        
                        m=folium.Map(location=[num_lat,num_lng],zoom_start=12) # 여기에 원하는 위도, 경도를 입력
                        markercluster=plugins.MarkerCluster()
                        # CGV 지점에 대한 정보가 저장되어 있는 데이터베이스를 사용하여
                        # 위도, 경도, CGV 지점 이름 값을 가져온다
                        lat_lng = cursor.execute("SELECT lat,lng FROM CGV").fetchall()
                        name = cursor.execute("SELECT name FROM CGV").fetchall()
                        
                        img = 'cgv.png'
                        
                        folium.Marker([num_lat, num_lng],tooltip='현재 위치').add_to(m)
                        
                        # for문 내에서 cgv 로고로 만든 커스텀 아이콘 객체를 형성
                        # marker를 만들 때 커스텀 아이콘을 사용하여 만들고
                        # marker 위치는 가져온 위도, 경도 값, tooltip은 가져온 지점 이름으로 지정
                        for i,j in zip(lat_lng,name):
                            icon_cgv = CustomIcon(
                            img,
                            icon_size = (40,20)
                            )
                            folium.Marker(i,tooltip=j[0],icon=icon_cgv).add_to(markercluster)
                        markercluster.add_to(m)
                        
                        a = cursor.execute(f"SELECT lat,lng FROM CGV where name='{place}'").fetchall()
                        
                        Movie.navigate(num_lat,num_lng,a[0][0],a[0][1],m) # 처음 두 자리에 사용자의 위치를 입력
                        
                        # 함수 혹은 클래스 내부에서 명령어를 사용하더라도 지도를 보여주게 하기 위해서
                        # html 형태로 지도를 저장하고 webbrowser.open 명령어를 통해서 지도를 웹브라우저로 호출
                        m.save('CGV_map.html')
                        
                        webbrowser.open('CGV_map.html')
                        
                    elif num_CGV == 3:
                        break
            
            # 이후에는 CGV의 구동 방식과 비슷하다
            elif num == 2:
                while(True):
                    print(m_Mega)
                    num_Mega = int(input('번호를 입력하세요 : '))
                    if num_Mega == 1:
                        MEGA.show_movie()
                        yn = input('영화 댓글 정보를 확인하시겠습니까? (y/n) : ')
                        if yn == 'y':
                            MEGA.find_url()
                            MEGA.find_review()
                            while(True):
                                print(mini_m1)
                                choice_Mega = int(input('번호를 입력하세요 : '))
                                if choice_Mega == 1:
                                    MEGA.charm() # 메가박스에는 매력 포인트 그래프를 그리는 기능도 보유하고 있다
                                elif choice_Mega == 2:
                                    MEGA.emotion()
                                elif choice_Mega == 3:
                                    break
                                    
                    elif num_Mega == 2:
                        while(True):
                            loc = input('현재 위치를 입력하세요 : ')
                            Movie.curr_loc(loc)
                            if check == 1:
                                print('위치 정보가 올바르지 않습니다')
                            elif check == 0:
                                break
                        
                        place = input('방문할 MEGABOX 지점 이름을 입력하세요 : ')
                        
                        m=folium.Map(location=[num_lat,num_lng],zoom_start=12) # 여기에 원하는 위도, 경도를 입력
                        markercluster=plugins.MarkerCluster()
                        lat_lng = cursor.execute("SELECT lat,lng FROM MEGA").fetchall()
                        name = cursor.execute("SELECT name FROM MEGA").fetchall()
                        
                        img = 'megabox.png'
                        
                        folium.Marker([num_lat, num_lng],tooltip='현재 위치').add_to(m)
                        
                        for i,j in zip(lat_lng,name):
                            icon_megabox = CustomIcon(
                            img,
                            icon_size = (120,60)
                            )
                            
                            folium.Marker(i,tooltip=j[0],icon=icon_megabox).add_to(markercluster)
                        markercluster.add_to(m)
                        
                        a = cursor.execute(f"SELECT lat,lng FROM MEGA where name='{place}'").fetchall()
                        
                        Movie.navigate(num_lat,num_lng,a[0][0],a[0][1],m) # 처음 두 자리에 사용자의 위치를 입력
                        
                        m.save('MEGA_map.html')
                        
                        webbrowser.open('MEGA_map.html')
                        
                    elif num_Mega == 3:
                        break
                        
            elif num == 3:
                while(True):
                    print(m_Lotte)
                    num_Lotte = int(input('번호를 입력하세요 : '))
                    if num_Lotte == 1:
                        LOTTE.show_movie()
                        yn = input('영화 댓글 정보를 확인하시겠습니까? (y/n) : ')
                        if yn == 'y':
                            LOTTE.find_url()
                            LOTTE.find_review()
                            while(True):
                                print(mini_m2)
                                choice_Lotte = int(input('번호를 입력하세요 : '))
                                if choice_Lotte == 1:
                                    LOTTE.emotion()
                                elif choice_Lotte == 2:
                                    break
                                    
                    elif num_Lotte == 2:
                        while(True):
                            loc = input('현재 위치를 입력하세요 : ')
                            Movie.curr_loc(loc)
                            if check == 1:
                                print('위치 정보가 올바르지 않습니다')
                            elif check == 0:
                                break

                        place = input('방문할 LOTTE CINEMA 지점 이름을 입력하세요 : ')
                        
                        m=folium.Map(location=[num_lat,num_lng],zoom_start=12) # 여기에 원하는 위도, 경도를 입력
                        markercluster=plugins.MarkerCluster()
                        lat_lng = cursor.execute("SELECT lat,lng FROM LOTTE").fetchall()
                        name = cursor.execute("SELECT name FROM LOTTE").fetchall()
                        
                        img = 'lotte.png'
                        
                        folium.Marker([num_lat, num_lng],tooltip='현재 위치').add_to(m)
                        
                        for i,j in zip(lat_lng,name):
                            icon_lotte = CustomIcon(
                            img,
                            icon_size = (45,45)
                            )
                            
                            folium.Marker(i,tooltip=j[0],icon=icon_lotte).add_to(markercluster)
                        markercluster.add_to(m)
                        
                        a = cursor.execute(f"SELECT lat,lng FROM LOTTE where name='{place}'").fetchall()
                        
                        Movie.navigate(num_lat,num_lng,a[0][0],a[0][1],m) # 처음 두 자리에 사용자의 위치를 입력
                        
                        m.save('LOTTE_map.html')
                        
                        webbrowser.open('LOTTE_map.html')
                    
                    elif num_Lotte == 3:
                        break

            elif num == 4:
                while(True):
                    print(m_Daum)
                    num_Daum = int(input('번호를 입력하세요 : '))
                    if num_Daum == 1:
                        DAUM.show_movie()
                        yn = input('영화 댓글 정보를 확인하시겠습니까? (y/n) : ')
                        if yn == 'y':
                            DAUM.find_url()
                            DAUM.find_review()
                            while(True):
                                print(mini_m2)
                                choice_Daum = int(input('번호를 입력하세요 : '))
                                if choice_Daum == 1:
                                    DAUM.emotion()
                                elif choice_Daum == 2:
                                    break
                    # 다음 영화는 영화관이 아니기 때문에 위치 정보를 통해 지도를 보여줄 수 없다
                    # 그래서 사용자가 2를 선택했을 때 지도 정보가 아닌 break를 통해서 while문을 빠져나가게 하였다
                    elif num_Daum == 2:
                        break
                        
            elif num == 5:
                con.close()
                
                break

In [178]:
movie = Movie()
movie.menu()

-----------------------------------------------------
                    1. CGV 정보                      
                  2. MEGABOX 정보   
                3. LOTTE CINEMA 정보
                 4. DAUM 영화 정보
                      5. 종료                      
-----------------------------------------------------
번호를 입력하세요 : 5
